In [ ]:
import pandas as pd
import pysolr


In [ ]:
# url = "http://b2find.eudat.eu/solr/select"
url = "http://localhost:8983/solr/ckan"


## Solr date range field

https://solr.apache.org/guide/6_6/working-with-dates.html

In [ ]:
facet = "extras_TempCoverage"

solr_params = {
    'echoParams': 'none',
    'rows': 0,
    'wt': 'json',
    'q': '*',
    #'fq': f"{{!field f={facet} op=Contains}}[1500 TO 2500]",
    'facet': 'true',
    'facet.range':[
        facet, 
    ],
    'facet.range.start': '-1000-01-01T00:00:00Z/YEAR',
    'facet.range.end': '2300-12-31T23:59:59Z/YEAR',
    'facet.range.gap': '+1YEARS',
}

In [ ]:
solr = pysolr.Solr(url)

In [ ]:
import time
run_start = time.time()

results = solr.search(**solr_params)

run_end = time.time() - run_start
run_end

In [ ]:
# results.facets

In [ ]:
result_counts = results.facets["facet_ranges"][facet]["counts"]
x = result_counts[0::2]
y = result_counts[1::2]


In [ ]:
df = pd.DataFrame(list(zip(x, y)), columns =['date', 'counts'])
df

In [ ]:
#df["date"]=df["date"].str.split(pat="-01-01T", n=1, expand=True)
df["date"]=df["date"].str[:-16]
df["date"]=pd.to_numeric(df["date"], downcast="signed")
df

In [ ]:
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure

source = ColumnDataSource(df)

p = figure(plot_height=300, plot_width=800, tools="xpan", toolbar_location=None,
           # x_axis_type="datetime", 
           x_axis_location="above",
           background_fill_color="#efefef", x_range=(df.date.loc[len(x)-460], df.date.loc[len(x)-260]))

p.line('date', 'counts', source=source)
p.yaxis.axis_label = 'Number of Datasets per Year'

select = figure(title="Drag the middle and edges of the selection box to change the range above",
                plot_height=130, plot_width=800, y_range=p.y_range,
                #x_axis_type="datetime", 
                y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('date', 'counts', source=source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

show(column(p, select))